In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


25/02/28 09:22:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sql("SHOW DATABASES IN demo").show()


+---------+
|namespace|
+---------+
|      nyc|
+---------+



In [3]:
# Get list of tables in the 'nyc' database
tables = spark.sql("SHOW TABLES IN nyc")
tables.show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|      nyc|           taxis_100|      false|
|      nyc|     taxis_100M_time|      false|
|      nyc|taxis_1000_50COLUMNS|      false|
|      nyc|            taxis_1B|      false|
|      nyc|taxis_100_50COLUM...|      false|
|      nyc|taxis_100000_50CO...|      false|
|      nyc|            taxis_1L|      false|
|      nyc|          taxis_1000|      false|
|      nyc|            taxis_10|      false|
|      nyc|   taxis_1L_5COLUMNS|      false|
|      nyc|         taxis_10000|      false|
|      nyc|            taxis_1M|      false|
|      nyc|          taxis_1L_5|      false|
|      nyc|          taxis_10_M|      false|
|      nyc|taxis_1000_50COLU...|      false|
|      nyc|  taxis_10_50COLUMNS|      false|
|      nyc|           taxis_10K|      false|
|      nyc|            taxis_1K|      false|
|      nyc|           taxis_10L|      false|
|      nyc

In [4]:
df=spark.sql("DESCRIBE demo.nyc.taxis_10000_50COLUMNS")
df.show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
| extra_col_0|   string|   NULL|
| extra_col_1|      int|   NULL|
| extra_col_2|   string|   NULL|
| extra_col_3|     date|   NULL|
| extra_col_4|   string|   NULL|
| extra_col_5|      int|   NULL|
| extra_col_6|   string|   NULL|
| extra_col_7|     date|   NULL|
| extra_col_8|   string|   NULL|
| extra_col_9|      int|   NULL|
|extra_col_10|   string|   NULL|
|extra_col_11|     date|   NULL|
|extra_col_12|   string|   NULL|
|extra_col_13|      int|   NULL|
|extra_col_14|   string|   NULL|
|extra_col_15|     date|   NULL|
|extra_col_16|   string|   NULL|
|extra_col_17|      int|   NULL|
|extra_col_18|   string|   NULL|
|extra_col_19|     date|   NULL|
+------------+---------+-------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F
import time

# Start the timer
start_time = time.time()

# Specify partitioning column, for example 'extra_col_3' (assuming it's a date column)
partition_column = "extra_col_3"

# Perform partitioning and save the partitioned table
spark.sql("""
    CREATE TABLE demo.nyc.taxis_10000_50COLUMNS_partitioned
    USING iceberg
    PARTITIONED BY ({} )
    AS
    SELECT * FROM demo.nyc.taxis_10000_50COLUMNS
""".format(partition_column))

# End the timer
end_time = time.time()

# Calculate the time taken for partitioning operation
execution_time = end_time - start_time
print(f"Time taken for partitioning operation: {execution_time} seconds")


25/02/28 09:28:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/02/28 09:28:19 ERROR Utils: Aborting task                        (0 + 4) / 4]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.ByteArrayOutputStream.<init>(ByteArrayOutputStream.java:79)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory$HeapBytesCompressor.<init>(CodecFactory.java:158)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory.createCompressor(CodecFactory.java:219)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory.getCompressor(CodecFactory.java:202)
	at org.apache.iceberg.parquet.ParquetWriter.<init>(ParquetWriter.java:90)
	at org.apache.iceberg.parquet.Parquet$WriteBuilder.build(Parquet.java:352)
	at org.apache.iceberg.parquet.Parquet$DataWriteBuilder.build(Parquet.java:734)
	at org.apache.iceberg.data.BaseFileWri